# Objective
In this analysis, I use a longitudinal cohort of household heads being followed over maximum of 12 years to estimate the income effect of obesity after adjusting for demographic and socio-economic status. I use longitudinal fixed effect model to estimate the effect. 

In [108]:
#import the libraries
import pandas as pd
import pandasql as ps
#display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [109]:
import os
import sys

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

import textwrap
from IPython.display import display

from patsy import dmatrices
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score

import statsmodels.formula.api as smf

In [110]:
# Enable inline plotting for graphics
%matplotlib inline

In [111]:
# Get Version information
print(textwrap.fill(sys.version),'\n')
print("Pandas version: {0}".format(pd.__version__),'\n')
print("Matplotlib version: {0}".format(matplotlib.__version__),'\n')
print("Numpy version: {0}".format(np.__version__),'\n')
print("Seaborn version: {0}".format(sns.__version__),'\n')

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 

Pandas version: 1.3.4 

Matplotlib version: 3.5.0 

Numpy version: 1.20.3 

Seaborn version: 0.11.1 



In [112]:
# So all output comes through from Ipython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [113]:
# Working Directory
print(os.getcwd())

C:\Users\run1\OneDrive - CDC\PSID New\J311955_Core_Family_2007_19


In [114]:
# Set New Working Directory 
# the try except tells Python to try to run the code
# and if an error occurs, print a message
#try:
os.chdir(r"C:\Users\run1\OneDrive - CDC\PSID New\J311955_Core_Family_2007_19")
#except:
 #   print("ERROR: You probably forgot to change the directory path!")

In [115]:
print("My working directory:\n" + os.getcwd())

My working directory:
C:\Users\run1\OneDrive - CDC\PSID New\J311955_Core_Family_2007_19


In [116]:
df = pd.read_stata("Final_Working_Clean_Family_Indi_Heads_Subsample_2007_19.dta", convert_categoricals=False)
#df.read()
#print(df.value_labels())

In [117]:
with pd.option_context("display.max_columns", None):
    display(df)

,PID,Year,UniqueInterviewID,Current_State,Institution_Type,Unique1968FID,Number_in_FU,NumberChildreninFU,WIC,FoodStamp,Whether_TANF_HD,Whether_SSI_HD,Whether_OtrWelfare_HD,HeadAndWifeTaxableInc,Total_Family_Income,ReleaseNumber,Fam_CompChange_Year,HeadAge,GenderHD,SpouseAge,Health_Status_HD,Heavy_Physical_Act_HD,Light_Physical_Act_HD,Strength_Physical_Act_HD,AgeFirstSmoked,WTR_Smoked,AgeLastSmoked,WTR_Drink,Num_DrinkPD,HeadWeight,HeadHeightFeet,HeadHeightInch,HeadHeightM,Health_Status_Spouse,HeadSpouseWeight,HeadSpouseHeightFeet,HeadSpouseHeightInch,HeadSpouseHeightM,OtherFamily_PoorHealth,HeadSpouseSpanish,HeadSpouseRace,HeadSpouseM2Race,HeadSpouseEthnicity,HeadSpanish,HeadM1Race,HeadM2Race,HeadEthnicity,HealthcareExpenditure,HealthinsExpenditure,CompEduHead,CompEduHeadSpouse,SEQ,REL,IndiAge,BirthYear,MaritalPair_Indi,Respondent_ind,InterviewHelp,Employment_Status,Edu_Years,Health_Good,Health_Ins_Kids_Type,Prev_24M_Months_NotCovered,Prev_12M_Months_NotCovered,WTR_Ins,Health_Ins_M1_Type,Health_Ins_M2_Type,Health_Ins_M3_Type,OFUM_Total_Taxable_Inc_Imp,FormationMonth_SO,SO_Fam_Year,FamID_for_SO,Follow_Status,Longitudinal_WT,Crosssectional_WT,Prev_Health_Ins_M1_Type,Prev_Health_Ins_M2_Type,Prev_Health_Ins_M3_Type,Prev_Health_Ins_M4_Type,Prev_12M_Months_Covered,Prev_24M_Months_Covered,GenderHDSpouse,Head,HeadSpouse,Term,lweight,Health_Insurance,Health_Insurance_numeric,Health_Insurance2,Health_Insurance3,Head_Height_F,Head_Height_I,Head_Height_M,Head_Height,Head_Weight,Head_BMI,Spouse_Height_F,Spouse_Height_I,Spouse_Height_M,Spouse_Height,Spouse_Weight,Spouse_BMI,Head_Weight_Cat,Spouse_Weight_Cat,Single,Self_Respondent,Head_AgeGroup2,Spouse_AgeGroup2,HeadRace,NoCollege,NoCollegeCouple,HeadEducation,HeadSpouseEducation,Food_Stamp,WIC_if_eligible,any_benefit,Obesity,allwaves,Children,HH_Education,Head_AgeGroup3,Spouse_AgeGroup3,Ever_Smoked_Cigarettes,Never_Smoked,Smoking_duration,Sample_Stratum,Sample_Cluster,Physical_Activity,Physical_Activity_Times,Physical_Activity_Level,bmi1,bmi2,bmi3,bmi_cat1,bmi_cat2,bmi_cat3,bmi_cat4,bmi_cat5,Previous_BMI_Cat,CPI,Pregnant,Pregnancy_in_HH,inf_adj_tfi,inf_adj_hwi,inf_adj_hce,inf_adj_hie,log_tfi,log_hwi,log_hce,log_hie,per_capita_total_income,log_pctfi,Employment_status_n,_est_feologit_4,_est_feologit_3_1,_est_feologit_3_2,_est_feologit_3_3,Age_at_First_Term,Head_AgeFirstTerm,Spouse_Age_at_First_Term,Spouse_AgeFirstTerm,Race_at_First_Term,Sex_at_First_Term,Race,Race_n,Age,Age_Square,_est_feologit_1,_est_feologit_2,_est_feologit_3,_est_t1,race1,race2,race3,race4,race5,_est_r1,_est_r2
0,4003,2007.0,148.0,47.0,0,4.0,3.0,1.0,5,5,5,5,5,23000,89560,7,0,55.0,1,53.0,2,0.0,0.0,5.0,0.0,5,0.0,0,0.0,190.0,5,5.0,NaN,5,185.0,5,6.0,NaN,5,0,1,0,5,0,1,0,5,2498,500,9.0,10.0,1.0,10.0,55.0,1951.0,1,0,NaN,3,9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0,1,43.264999,17411,5.0,0.0,0.0,0.0,12.0,12.0,NaN,1.0,0.0,1.0,48.154999,,NaN,,,5.0,5.0,NaN,1.6510,190.0,31.617304,5.0,6.0,NaN,1.6764,185.0,29.859453,4.0,3.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,4.0,0.0,1.0,0.0,31.0,2,1.0,5.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,1.2435,NaN,0.0,111367.859375,28600.500000,3106.262939,621.750000,11.620603,10.261214,8.041497,6.434145,37122.621094,10.522009,3.0,0,0,0,0,55.0,2.0,53.0,2.0,1.0,1.0,1.0,1.0,56.0,3136.0,1,1,0,0,1.0,0.0,0.0,0.0,0.0,0,1
1,4003,2009.0,13.0,47.0,0,4.0,3.0,0.0,5,1,5,5,5,0,24960,5,0,57.0,1,55.0,3,0.0,0.0,0.0,0.0,5,0.0,1,3.0,185.0,5,6.0,NaN,5,200.0,5,6.0,NaN,5,0,1,0,5,0,1,0,5,0,0,10.0,11.0,1.0,10.0,57.0,1951.0,1,0,0.0,4,10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0,1,43.264999,17496,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,2.0,48.154999,,NaN,,,5.0,6.0,NaN,1.6764,185.0,29.859453,5.0,6.0,NaN,1.6764,200.0,32.280491,3.0,4.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,5.0,0.0,1.0,0.0,31.0,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.1921,NaN,0.0,29754.816406,0.000000,0.000000,0.000000,10.300780,0.000000,0.000000,0.000000,9918.272461,9.202235,4.0,0,0,0,0,55.0,2.0,53.0,2.0

In [118]:
#df = df.set_index(["PID", "Year"])

In [119]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78579 entries, 0 to 78578
Columns: 178 entries, PID to _est_r2
dtypes: float32(113), float64(13), int32(8), int8(41), object(3)
memory usage: 49.5+ MB


,PID,Year,UniqueInterviewID,Current_State,Institution_Type,Unique1968FID,Number_in_FU,NumberChildreninFU,WIC,FoodStamp,Whether_TANF_HD,Whether_SSI_HD,Whether_OtrWelfare_HD,HeadAndWifeTaxableInc,Total_Family_Income,ReleaseNumber,Fam_CompChange_Year,HeadAge,GenderHD,SpouseAge,Health_Status_HD,Heavy_Physical_Act_HD,Light_Physical_Act_HD,Strength_Physical_Act_HD,AgeFirstSmoked,WTR_Smoked,AgeLastSmoked,WTR_Drink,Num_DrinkPD,HeadWeight,HeadHeightFeet,HeadHeightInch,HeadHeightM,Health_Status_Spouse,HeadSpouseWeight,HeadSpouseHeightFeet,HeadSpouseHeightInch,HeadSpouseHeightM,OtherFamily_PoorHealth,HeadSpouseSpanish,HeadSpouseRace,HeadSpouseM2Race,HeadSpouseEthnicity,HeadSpanish,HeadM1Race,HeadM2Race,HeadEthnicity,HealthcareExpenditure,HealthinsExpenditure,CompEduHead,CompEduHeadSpouse,SEQ,REL,IndiAge,BirthYear,MaritalPair_Indi,Respondent_ind,InterviewHelp,Employment_Status,Edu_Years,Health_Good,Health_Ins_Kids_Type,Prev_24M_Months_NotCovered,Prev_12M_Months_NotCovered,WTR_Ins,Health_Ins_M1_Type,Health_Ins_M2_Type,Health_Ins_M3_Type,OFUM_Total_Taxable_Inc_Imp,FormationMonth_SO,SO_Fam_Year,FamID_for_SO,Follow_Status,Longitudinal_WT,Crosssectional_WT,Prev_Health_Ins_M1_Type,Prev_Health_Ins_M2_Type,Prev_Health_Ins_M3_Type,Prev_Health_Ins_M4_Type,Prev_12M_Months_Covered,Prev_24M_Months_Covered,GenderHDSpouse,Head,HeadSpouse,Term,lweight,Health_Insurance_numeric,Head_Height_F,Head_Height_I,Head_Height_M,Head_Height,Head_Weight,Head_BMI,Spouse_Height_F,Spouse_Height_I,Spouse_Height_M,Spouse_Height,Spouse_Weight,Spouse_BMI,Head_Weight_Cat,Spouse_Weight_Cat,Single,Self_Respondent,Head_AgeGroup2,Spouse_AgeGroup2,HeadRace,NoCollege,NoCollegeCouple,HeadEducation,HeadSpouseEducation,Food_Stamp,WIC_if_eligible,any_benefit,Obesity,allwaves,Children,HH_Education,Head_AgeGroup3,Spouse_AgeGroup3,Ever_Smoked_Cigarettes,Never_Smoked,Smoking_duration,Sample_Stratum,Sample_Cluster,Physical_Activity,Physical_Activity_Times,Physical_Activity_Level,bmi1,bmi2,bmi3,bmi_cat1,bmi_cat2,bmi_cat3,bmi_cat4,bmi_cat5,Previous_BMI_Cat,CPI,Pregnant,Pregnancy_in_HH,inf_adj_tfi,inf_adj_hwi,inf_adj_hce,inf_adj_hie,log_tfi,log_hwi,log_hce,log_hie,per_capita_total_income,log_pctfi,Employment_status_n,_est_feologit_4,_est_feologit_3_1,_est_feologit_3_2,_est_feologit_3_3,Age_at_First_Term,Head_AgeFirstTerm,Spouse_Age_at_First_Term,Spouse_AgeFirstTerm,Race_at_First_Term,Sex_at_First_Term,Race,Race_n,Age,Age_Square,_est_feologit_1,_est_feologit_2,_est_feologit_3,_est_t1,race1,race2,race3,race4,race5,_est_r1,_est_r2
count,7.857900e+04,78579.000000,78579.000000,78579.000000,78579.0,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,7.857900e+04,7.857900e+04,78579.000000,78579.000000,78579.000000,78579.000000,55249.000000,78579.000000,77814.000000,77613.000000,78139.000000,77837.000000,78579.000000,77908.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,57115.000000,78579.000000,78579.000000,78579.000000,78579.000000,57115.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,77044.000000,77253.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,78579.000000,68063.000000,78579.000000,78579.000000,78579.0,78579.000000,45921.000000,45921.000000,57053.000000,57115.000000,57115.000000,57115.000000,78579.0,78579.000000,78579.000000,78579.000000,78579.00000,78579.000000,78579.000000,32658.000000,32658.000000,32658.000000,32658.000000,21464.000000,21464.000000,34617.000000,78579.000000,78579.000000,78579.000000,78579.000000,57115.000000,77846.000000,67666.000000,258.000000,78104.000000,77501.000000,77273.000000,54657.000000,51881.000000,186.000000,54841.000000,52965.000000,52805.000000,77273.000000,52805.000000,78579.000000,78579.000000,78578.000000,55249.000000,77936.000000,77044.000000,78579.000000,77044.000000,78579.000000,78488.000000,43953.000000,78579.000000,78579.000000,78579.000000,78579.000000

In [120]:
## Set default figure size to be larger
## this may only work in matplotlib 2.0+!

matplotlib.rcParams['figure.figsize'] = [10.0, 6.0]
matplotlib.rcParams['font.serif'] = "Georgia"
matplotlib.rcParams['font.family'] = "serif"

## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

In [121]:
# Get fixed effect regression for per capit family income using just year effect.
import statsmodels.formula.api as smf

mod = smf.ols("log_tfi ~ C(Year)", data=df).fit()


In [122]:
mod.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,11.0574,0.013,837.489,0.000,11.032,11.083
C(Year)[T.2009.0],0.0039,0.018,0.211,0.833,-0.032,0.040
C(Year)[T.2011.0],-0.1274,0.018,-6.928,0.000,-0.163,-0.091
C(Year)[T.2013.0],-0.1559,0.018,-8.501,0.000,-0.192,-0.120
C(Year)[T.2015.0],-0.1479,0.018,-8.033,0.000,-0.184,-0.112
C(Year)[T.2017.0],-0.1255,0.018,-6.852,0.000,-0.161,-0.090
C(Year)[T.2019.0],-0.1354,0.018,-7.504,0.000,-0.171,-0.100


In [123]:
# Check which variables are varying over time for each unique individual
df.groupby("PID").std().sum()

Year                          5.169004e+04
UniqueInterviewID             3.048020e+07
Current_State                 1.757781e+04
Institution_Type              0.000000e+00
Unique1968FID                 0.000000e+00
Number_in_FU                  6.824608e+03
NumberChildreninFU            5.258495e+03
WIC                           1.240663e+04
FoodStamp                     6.423736e+03
Whether_TANF_HD               2.158228e+04
Whether_SSI_HD                2.251105e+04
Whether_OtrWelfare_HD         2.137127e+04
HeadAndWifeTaxableInc         3.889021e+08
Total_Family_Income           4.102548e+08
ReleaseNumber                 2.781790e+04
Fam_CompChange_Year           4.521794e+03
HeadAge                       5.240183e+04
GenderHD                      5.979879e+02
SpouseAge                     3.495898e+04
Health_Status_HD              8.690024e+03
Heavy_Physical_Act_HD         2.950182e+04
Light_Physical_Act_HD         3.728222e+04
Strength_Physical_Act_HD      1.837623e+04
AgeFirstSmo

In [124]:
# Get mean values of inflation adjusted total family per capita income
df.groupby(by=['PID', 'Year'])["per_capita_total_income"].mean()

PID      Year  
4003     2007.0     37122.621094
         2009.0      9918.272461
         2011.0     23097.744141
4004     2007.0    342957.312500
         2009.0     85645.234375
                       ...      
6872184  2011.0     14859.000000
         2013.0     13273.356445
         2015.0     15708.814453
         2017.0     12840.754883
         2019.0     13857.142578
Name: per_capita_total_income, Length: 78579, dtype: float32

In [125]:
## Run Fixed Effects Model by demeaning the observations
Y = "log_tfi"
T = "Obesity"
X = [T, "Single","GenderHD","Age","Age_Square","any_benefit", "Edu_Years" ]
mean_data = df.groupby("PID")[X+[Y]].mean()
mean_data.head()

,Obesity,Single,GenderHD,Age,Age_Square,any_benefit,Edu_Years,log_tfi
PID,,,,,,,,
4003,0.333333,0.000000,1.000000,58.000000,3366.666748,0.333333,9.666667,10.887347
4004,0.000000,1.000000,2.000000,57.000000,3254.000000,0.000000,13.500000,11.842923
4006,0.000000,1.000000,2.000000,55.000000,3041.000000,1.000000,12.000000,10.156821
4007,0.000000,1.000000,2.000000,59.000000,3481.000000,0.000000,12.000000,11.018153
4031,0.500000,0.333333,1.333333,39.666668,1591.333374,0.833333,12.000000,10.140614


In [126]:
## Get the demeaned data
demeaned_data = (df
               .set_index("PID") # set the index as the person indicator
               [X+[Y]]
               - mean_data) # subtract the mean data

demeaned_data.head()

,Obesity,Single,GenderHD,Age,Age_Square,any_benefit,Edu_Years,log_tfi
PID,,,,,,,,
4003,0.666667,0.0,0.0,-2.0,-230.666748,-0.333333,-0.666667,0.733255
4003,-0.333333,0.0,0.0,0.0,-2.666748,0.666667,0.333333,-0.586567
4003,-0.333333,0.0,0.0,2.0,233.333252,-0.333333,0.333333,-0.146688
4004,0.000000,0.0,0.0,-3.0,-338.000000,0.000000,-1.500000,0.902441
4004,0.000000,0.0,0.0,-1.0,-118.000000,0.000000,0.500000,-0.484942


In [127]:
## Get the OLS estimates for time-demeaned data
mod = smf.ols(f"{Y} ~ {'+'.join(X)}", data=demeaned_data).fit()
mod.summary().tables[1]


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-1.671e-09,0.003,-6.12e-07,1.000,-0.005,0.005
Obesity,0.0172,0.011,1.499,0.134,-0.005,0.040
Single,-0.6288,0.028,-22.638,0.000,-0.683,-0.574
GenderHD,-0.1183,0.036,-3.280,0.001,-0.189,-0.048
Age,0.0802,0.003,27.095,0.000,0.074,0.086
Age_Square,-0.0008,2.85e-05,-29.018,0.000,-0.001,-0.001
any_benefit,-0.1622,0.013,-12.814,0.000,-0.187,-0.137
Edu_Years,-0.0003,0.000,-0.830,0.406,-0.001,0.000


In [128]:
# Cluster your standard errors by PID
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from formulaic import Formula

df = df.set_index(["PID", "Year"])

exog_vars = ["Obesity","Single","GenderHD","Age","Age_Square","any_benefit","Edu_Years"]
exog = sm.add_constant(df[exog_vars])
mod = PanelOLS(df['log_tfi'], exog, entity_effects=True)

result = mod.fit(cov_type='clustered', cluster_entity=True)
result.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
const,9.5643,0.1176,81.336,0.0000,9.3338,9.7948
Obesity,0.0172,0.0141,1.2180,0.2232,-0.0105,0.0449
Single,-0.6288,0.0454,-13.860,0.0000,-0.7177,-0.5398
GenderHD,-0.1183,0.0566,-2.0896,0.0367,-0.2292,-0.0073
Age,0.0802,0.0043,18.727,0.0000,0.0718,0.0886
Age_Square,-0.0008,4.196e-05,-19.733,0.0000,-0.0009,-0.0007
any_benefit,-0.1622,0.0208,-7.7931,0.0000,-0.2030,-0.1214
Edu_Years,-0.0003,0.0003,-0.9457,0.3443,-0.0009,0.0003


In [129]:
# Cluster your standard errors by PID and year
result2 = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
result2.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
const,9.5643,0.3780,25.306,0.0000,8.8235,10.305
Obesity,0.0172,0.0178,0.9646,0.3347,-0.0177,0.0522
Single,-0.6288,0.0346,-18.152,0.0000,-0.6966,-0.5609
GenderHD,-0.1183,0.0404,-2.9244,0.0035,-0.1975,-0.0390
Age,0.0802,0.0135,5.9560,0.0000,0.0538,0.1066
Age_Square,-0.0008,0.0001,-7.4179,0.0000,-0.0010,-0.0006
any_benefit,-0.1622,0.0176,-9.2080,0.0000,-0.1968,-0.1277
Edu_Years,-0.0003,0.0002,-1.2735,0.2028,-0.0008,0.0002


In this preliminary analysis, we find that Obesity has no significant effect on income. However, we do find significant association of gender, age, marital status and receiving any benefit with household income.